# 02 - Labeling Intent

---

### Intent Categories

| Intent Label        | Description                                      |
|---------------------|------------------------------------------------|
| `cancel_service`    | Customer wants to cancel account or switch plan|
| `billing_issue`     | Complaints or questions about billing           |
| `technical_issue`   | Problems with device, network, or service       |
| `account_help`      | Issues with account management or login         |
| `upgrade_request`   | Requests to upgrade device or plan               |
| `general_question`  | General questions or inquiries                    |
| `positive_feedback` | Praise or compliments                             |
| `complaint`         | Negative feedback not related to billing or tech|
| `other`             | Anything else or unclear                          |

# Labeling and Logic of Categorization

In [2]:
import pandas as pd
import sys
import os


# Fixing routing issue
project_root = os.path.abspath('..')
sys.path.append(project_root)

from scripts.preprocess import assign_intent

# Loads Data from Cleaned CSV
df = pd.read_csv("../data/processed/cleaned_tweets.csv")

# Sample 250 rows to label
golden_df = df.sample(250, random_state=42).copy()

# Creates Categories to put into column later
intent_categories = [
    "Billing",
    "Technical Support",
    "Account Management",
    "Complaint",
    "Praise/Thank You",
    "Other"
]

keywords = {
    "Billing": ["bill", "charge", "payment", "refund"],
    "Technical Support": ["error", "issue", "problem", "disconnect", "slow"],
    "Account Management": ["password", "login", "account", "reset"],
    "Complaint": ["bad", "terrible", "worst", "disappointed", "angry"],
    "Praise/Thank You": ["thank", "great", "love", "awesome", "appreciate"]
}

# Apply Categorization Function to Datarframe and Write to CSV file

In [5]:
df['intent'] = df['cleaned_text'].apply(lambda text: assign_intent(text, keywords))
print(df[['cleaned_text', 'intent']].sample(10))

df['tweet_length'] = df['cleaned_text'].str.len()

#print(df[['cleaned_text', 'tweet_length']].sample(10))
print(df[['cleaned_text', 'is_question']].sample(10))

# Writes golden dataset to csv file
df.to_csv("../data/processed/golden_intent_labeled.csv", index=False)
print("Golden dataset saved")

                                           cleaned_text              intent
7225   set to match the local market competition tabby                Other
2194         i need you to add a last name to my ticket               Other
8545   i can look into this can you send me the acco...  Account Management
3887                             but its not solved yet               Other
5426      here to help contact us via so we can connect               Other
1362   what do i have to do to stop omore texting me...               Other
1132                                      ahhhh thanks     Praise/Thank You
6109   ah got it your plan will be updated at the st...             Billing
2666   ich bitte darum mein freitag ist versaut hab ...               Other
1133   ah no worries andrew glad you got it sorted h...    Praise/Thank You
                                           cleaned_text  is_question
2300   we hope youre having a great time alex if you...            1
6080   ol andr com certeza

# Quick Evaluation

In [4]:
# Peep the examples by intent
for label in df['intent'].unique():
    print(f"Intent: {label}")
    sample = df[df['intent'] == label][['cleaned_text']].dropna().sample(3, random_state=1)
    for i, row in enumerate(sample.itertuples(index=False), 1):
        print(f"\n  {i}. {row.cleaned_text}")

Intent: Technical Support

  1.  we can certainly understand and would like to help you resolve this issue if youd like to continue troubleshooting please reply in dm 

  2.  do you have an email address i have problems with shipments and nobody will give me a straight answer about anything

  3.  hi carl thats a good idea lets define and solve a problem by using solver let us know if you have any feedback
Intent: Other

  1.  yesterday my household could stream multi devices today no whats going on and how do i fix it

  2.  only the best for you victor we hope we see you again soon

  3.  im sorry to know that wed like to take a closer look into this please drop in mp 
Intent: Praise/Thank You

  1.  message sent thanks

  2.  that would be so appreciated we love being platinum delta skymiles members

  3.  greetings we appreciate your interest in our products the variants that you see on the us website is with respect to the component upgrades you could check if the upgrades are ava

# Additional Features